#### Importing Libraries

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

#### Commomn Functions


In [0]:
class windows:
    def dense_rank(self,df, partition, order):
        df_dense_rank = df.withColumn("dense_rank", dense_rank().over(Window.partitionBy(partition).orderBy(desc(order))))
        return df_dense_rank
    
    def rank(self,df,partition,order):
        df_rank = df.withColumn("ranks", rank().over(Window.partitionBy(partition).orderBy(desc(order))))
        return df_rank
    
    def row_number(self,df,partition,order):
        df_row_number = df.withColumn("row_numbers", row_number().over(Window.partitionBy(partition).orderBy(order)))
        return df_row_number

In [0]:
#%run "/Workspace/Users/kolukulamanish29@gmail.com/Databricks ETE Project/silver_scripts/silver_common_functions"

#### Data Reading

In [0]:
df_orders = spark.readStream.format("delta") \
        .load("abfss://bronze@dldatabricks.dfs.core.windows.net/orders")

In [0]:
df_orders = df_orders.withColumnRenamed("_rescued_data", "rescued_data") \
        .withColumn('year', year(col('order_date').cast(DateType()))) \
        .drop("rescued_data")


     


In [0]:
# obj = windows()
# df=obj.dense_rank(df,'year','total_amount')
# df=obj.rank(df,'year','total_amount')
# df=obj.row_number(df,'year','total_amount')


In [0]:
# df_orders.display()

#### Data Writing

In [0]:
df_orders.writeStream.format('delta') \
          .outputMode('append') \
          .option("checkpointLocation", "abfss://silver@dldatabricks.dfs.core.windows.net/checkpoint_orders") \
          .option('path', 'abfss://silver@dldatabricks.dfs.core.windows.net/orders') \
          .trigger(once=True) \
          .start()